In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_csv('data/train_large.csv')


In [ ]:
rep = [  
        "Facility Id",
        "CCS Procedure Code",
        "CCS Diagnosis Code",
        "APR DRG Code",
        "APR MDC Code",
        "APR Severity of Illness Code",
        "Unnamed: 0"
]
to_drop = rep
# Features for polynomial
to_poly = [
        'Length of Stay','Birth Weight','Facility Name','APR DRG Description',
        'CCS Diagnosis Description', 'CCS Procedure Description',
        'Operating Certificate Number'
]
# Features for one hot encoding
to_onehot = [
        'APR Medical Surgical Description','APR Risk of Mortality', 'APR Severity of Illness Description','APR MDC Description',
        'Emergency Department Indicator','Health Service Area','Type of Admission',
        'Age Group', 'Gender', 'Race', 'Ethnicity',
        'Payment Typology 1', 'Payment Typology 2', 'Payment Typology 3','Patient Disposition','Hospital County','Zip Code - 3 digits'
]

In [ ]:
data = data.drop(to_drop,axis=1)

In [2]:
X = data.drop(['Total Costs'],axis = 1)
y = data['Total Costs']

In [ ]:
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
ohe = OneHotEncoder(drop = 'first',sparse = False)
poly = PolynomialFeatures(degree = 2,include_bias = False)
X_cont = poly.fit_transform(X[to_poly])
X_ohe = ohe.fit_transform(X[to_onehot])
b = np.ones((X_cont.shape[0],1))
X = np.c_[b,X_cont,X_ohe]

In [5]:
X.shape

(1600000, 31)

In [3]:
test = pd.read_csv('data/test.csv')

In [ ]:
test = test.drop(to_drop,axis = 1)
test_cont = poly.transform(test[to_poly])
test_ohe = ohe.transform(test[to_onehot])
b = np.ones((test_cont.shape[0],1))
test = np.c_[b,test_cont,test_ohe]

In [4]:
test.shape

(10000, 31)

In [6]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model = XGBRegressor(n_estmators = 50, learning_rate = 0.1, max_depth = 6, n_jobs = -1, random_state = 41,verbrosity = 2)


In [7]:
model.fit(X_train, y_train)

[19:37:41] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:573: 
Parameters: { "n_estmators", "verbrosity" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_estmators=50, n_jobs=-1, num_parallel_tree=1,
             random_state=41, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None, verbrosity=2)

In [8]:
model.score(X_test, y_test),model.score(X,y)

(0.8085614637281895, 0.8543131834074443)

In [ ]:
model.feature_importances_

In [9]:
output = model.predict(test)

In [10]:
np.savetxt("xgboost_r2-85.txt",output)

In [2]:
from sklearn.metrics import r2_score
import numpy as np
pred = np.loadtxt("WO/out_c.txt")
best = np.loadtxt("WO/xgboost_r2-85.txt")
r2_score(best,pred)

0.8456541873560363

0.7775842853565693